In [1]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Input, Concatenate, BatchNormalization, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

# Descargar la lista de stopwords si no está ya descargada
nltk.download('stopwords')

# Cargar las stopwords en español
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lingf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = Loader.load_train()

c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:133: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,
c:\Users\lingf\OneDrive\Escritorio\SRC\Cars_Price_Prediction-1\NLP\..\utils\loader.py:145: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [3]:
descriptions = [col for col in train.columns if "description" in col]

print(descriptions)

def custom_concat(row, cols):
    # Construir la descripción con lógica condicional basada en el valor de la celda
    parts = []
    for col_name in cols:  # Cambio para iterar solo sobre las columnas especificadas
        if col_name in row.index:  # Verificar que el nombre de la columna esté en el DataFrame
            value = row[col_name]
            if value == "no tiene" or not isinstance(value, str):
                parts.append(f"no tiene {col_name}")
            else:
                parts.append(str(value))  # Convertir a string para evitar problemas al unir
    # Unir todas las partes con espacios
    return ' '.join(parts)

# Aplicar la función al DataFrame
train['full_description'] = train.apply(custom_concat, axis=1, args=(descriptions,))
#train[descriptions].apply(lambda col: col.apply(lambda x: type(x).__name__).value_counts())

#descriptions.append("idx")
#descriptions.append("price")
#descriptions.append("km")

['motor_description', 'transmission_description', 'parking_sensors_description', 'Exterior_Llantas_description', 'Exterior_Molduras_description', 'Exterior_Neumáticos_description', 'Exterior_Puerta_description', 'Exterior_Retrovisor_description', 'Confort_Volante_description', 'seat_description', 'head_supporter_description']


In [4]:
filtered_columns = ["price", "km", "fuelType", "full_description"]
train = train[filtered_columns]
train["price_per_kilometer"] = train["price"]/train["km"]
train.dropna(inplace=True)
train

,price,km,fuelType,full_description,price_per_kilometer
idx,,,,,
105953,-0.077067,-0.219436,Gasolina,"Motor de 1,3 litros ( 1.332 cc ) , cuatro cili...",0.351204
15683,-0.193101,0.362457,Diésel,"Motor de 2,0 litros ( 1.969 cc ) , cuatro cili...",-0.532755
153684,-0.308670,0.531285,Diésel,"Motor de 1,6 litros ( 1.560 cc ) , cuatro cili...",-0.580988
156928,-0.058037,-0.439005,Híbrido,"Motor de 1,8 litros ( 1.798 cc ) , cuatro cili...",0.132202
24443,-0.935252,2.206234,Diésel,"Motor de 1,9 litros ( 1.910 cc ) , 4 cilindros...",-0.423913
...,...,...,...,...,...
19845,-0.058037,-0.113179,Diésel,"Motor de 1,6 litros ( 1.598 cc ) , cuatro cili...",0.512792
102652,-0.889071,0.848626,Gasolina,"Motor de 2,0 litros ( 1.997 cc ) , 4 cilindros...",-1.047659
101658,-0.309134,0.509224,Gas natural (CNG),"Motor de 1,4 litros ( 1.395 cc ) , cuatro cili...",-0.607070


In [5]:
#Parameters
verb_size = 200

In [6]:
def preprocess_text(text):
    # Convertir a minúsculas y eliminar caracteres especiales (pero mantener los números)
    text = text.lower()
    text = re.sub(r'[^\w\s\d]', '', text)  # Eliminar signos de puntuación, pero mantener números
    tokens = text.split()
    
    # Filtrar tokens, eliminando las stopwords
    filtered_tokens = [token for token in tokens if token not in spanish_stopwords]
    
    return filtered_tokens

train['tokens'] = train['full_description'].apply(preprocess_text)
model_w2v = Word2Vec(sentences=train['tokens'], vector_size=verb_size, window=1, min_count=3, workers=8)
word_vectors = model_w2v.wv

def get_average_embedding(tokens, model):
    embeddings = [model[word] for word in tokens if word in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size) 
train['embedding'] = train['tokens'].apply(lambda x: get_average_embedding(x, word_vectors))

In [7]:
# Load data (assuming df is your DataFrame containing the required columns)
texts = train['full_description'].values  # descriptions
embeddings = train['embedding'].values  # embeddings

# Tokenization and sequence padding
tokenizer = Tokenizer(num_words=verb_size)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences,  maxlen=verb_size, padding='post')

# Split data into training and testing sets
train_data, test_data, train_embeddings, test_embeddings, train_prices, test_prices = train_test_split(
    data, embeddings, train['price'].values, test_size=0.2, random_state=42, stratify=train['fuelType'])

# Normalize embeddings
train_embeddings = tf.keras.utils.normalize(train_embeddings)
test_embeddings = tf.keras.utils.normalize(test_embeddings)
train_embeddings = np.transpose(train_embeddings)
test_embeddings = np.transpose(test_embeddings)

In [8]:
def process_embedding(embed):
    standard_length = 200
    flat_embed = np.concatenate([np.array(e).flatten() for e in embed])  # Flatten
    # Pad with zeros or truncate to the standard length
    if len(flat_embed) > standard_length:
        return flat_embed[:standard_length]
    elif len(flat_embed) < standard_length:
        return np.pad(flat_embed, (0, standard_length - len(flat_embed)), mode='constant')
    return flat_embed

train_embeddings = np.array([process_embedding(e) for e in train_embeddings], dtype=np.float32)
train_embeddings = tf.convert_to_tensor(train_embeddings, dtype=tf.float32)
train_data = tf.convert_to_tensor(train_data, dtype=tf.int32)
train_prices = tf.convert_to_tensor(train_prices, dtype=tf.float32)

ValueError: setting an array element with a sequence.

In [23]:
# Text input branch
text_input = Input(shape=(None,), dtype='int32')
text_embed_layer = Embedding(input_dim=verb_size, output_dim=verb_size, input_length=train_data.shape[1])
text_embed = text_embed_layer(text_input)
lstm_out = LSTM(128)(text_embed)

c:\Users\lingf\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [24]:
# Embedding input branch
embed_input = Input(shape=(train_embeddings.shape[1],))
dense_embed = Dense(128, activation='relu')(embed_input)

# Concatenate both branches
concat = Concatenate()([lstm_out, dense_embed])
hidden = Dense(256, activation='relu')(concat)
output = Dense(1, activation='linear')(hidden)  # Output layer for price prediction

model = Model(inputs=[text_input, embed_input], outputs=output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, None, 200) │     40,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 128)       │    168,448 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     25,728 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256)       │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     65,792 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │        257 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 300,225 (1.15 MB)

 Trainable params: 300,225 (1.15 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
print(train_data.shape)      # Should be (number of samples, sequence length)
print(train_embeddings.shape)

(92477, 200)
(92477, 200)


In [26]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Train the model
model.fit([train_data, train_embeddings], train_prices,
          validation_data=([test_data, test_embeddings], test_prices),
          epochs=250, batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
test_loss, test_mse = model.evaluate([test_data, test_embeddings], test_prices)
print(f'Test MSE: {test_mse}')